# Async TreeSummarizeQuery Demo

In [1]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes. 
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.  
import nest_asyncio
nest_asyncio.apply()

In [2]:
import time
from llama_index import ListIndex, SimpleDirectoryReader

In [3]:
query_str = "What is Paul Graham's biggest achievement?"

In [4]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [ ]:
index = ListIndex.from_documents(documents)

#### By default, generate a response through hierarchical tree summarization (i.e., `response_mode=tree_summarize`) makes blocking LLM calls

In [ ]:
start_time = time.perf_counter()
query_engine = index.as_query_engine(response_mode='tree_summarize')
query_engine.query(query_str)
elapsed_time = time.perf_counter() - start_time

It takes ~21s to generate a response through hierarchical tree summarization (i.e., `response_mode=tree_summarize`).

#### Option 1: Running `aquery` (async query call) will take advantage of async LLM calls

In [ ]:
import asyncio
start_time = time.perf_counter()
task = query_engine.aquery(query_str)
asyncio.run(task)
elapsed_time = time.perf_counter() - start_time

It takes ~6.8s to generate a response through `aquery` (i.e., `response_mode=tree_summarize`).

#### Option 2: Pass in `use_async=True` to enable asynchronous LLM calls within a synchronous `query`

This approach makes a synchronous `query` calls, but runs async tasks during the "tree_summarize" operation.

In [ ]:
start_time = time.perf_counter()
query_engine = index.as_query_engine(
    response_mode='tree_summarize', 
    use_async=True,
)
query_engine.query(query_str)
elapsed_time = time.perf_counter() - start_time

It takes ~6.9s to generate a response through hierarchical tree summarization (i.e., `response_mode=tree_summarize`).